In [39]:
!pip install fuzzywuzzy[speedup]

     ------------------------------------ 101.3/101.3 kB 835.8 kB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 3.9 MB/s eta 0:00:00


In [40]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import os
from os.path import join
from fuzzywuzzy import process

In [2]:
def str_to_path(path_str):
    '''Converts windows path string to os.path'''
    path_list = path_str.split('\\')
    path = os.path.join(path_list[0], os.sep, *path_list[1:])
    return path

In [89]:
path = str_to_path(r"N:\WATER_Program\Groundwater\Global_GDE_Map\data")
plots_in = os.path.join(path, "sPlotOpen", "sPlotOpen_header(2).txt")
veg_in = os.path.join(path, "sPlotOpen", "sPlotOpen_DT(1).txt")
tbl_in = str_to_path(r"N:\WATER_Program\Groundwater\Global_GDE_Map\tables\sPlotOpen_GDE_review_20230407.csv")
fuzzy_out = str_to_path(r"N:\WATER_Program\Groundwater\Global_GDE_Map\tables\sPlotOpen_fuzzy_match.csv")
tbl_out = str_to_path(r"N:\WATER_Program\Groundwater\Global_GDE_Map\tables\sPlotOpen_GDE_points_20230407.csv")

In [66]:
df = pd.read_csv(tbl_in)
df = df.rename(columns={'GDE_Class(2=not groundwater-dependent;1= groundwater-dependent)': 'GDE_Class', 'GDE_Certainty_(1=High;2=Medium;3=Low)': 'GDE_Cert'})
df

,Species,Continent,Count_plots,GDE_Class,GDE_Cert,References,ExpertName,ExpertEmail,Notes,Species_Kristina_values,Species_sPlot_values,Species_Kristina,Species_sPlot
0,/new species,Asia,1,1.0,2.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,NaN,NaN,NaN,NaN,NaN
1,Abarema adenophora,Asia,2,1.0,1.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,"restricted to desert areas in central Asia, Af...",Abarema adenophora/rare species,Abarema adenophora,NaN,NaN
2,Abelia corymbosa,Asia,2,1.0,1.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,Native to Central Asia,NaN,NaN,NaN,NaN
3,Abelia mosanensis,Asia,1,1.0,2.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,evergreen shrubs are native to eastern Asia (J...,NaN,NaN,NaN,NaN
4,Abies alba,Europe,2113,1.0,2.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56045,Zygophyllum teretifolium,Africa,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56046,Zygophyllum xanthoxylum,Asia,68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56047,Zygostigma australe,South America,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56048,NaN,Africa,632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
#Error Checks on review table
print(df.GDE_Class.unique().tolist())
print(df.GDE_Cert.unique().tolist())
dfa = (df.groupby(['GDE_Class', 'GDE_Cert'], dropna=False)
           .agg({'Species':'count', 'Count_plots':'sum'})
           .reset_index()
           .rename(columns={'Species':'Count_species'})
          )
print(dfa)
# Get only the species with GDE class data
dfb = df.dropna(subset=['GDE_Class'])
print(len(df))

print(dfb.Continent.unique().tolist())
print(dfb[dfb.duplicated(['Species','Continent'], keep=False)].sort_values(by=['Species'], ascending=[True]))
#Remove any duplicate rows based on a subset of rows
dfb = dfb.drop_duplicates(subset=['Species','Continent'], keep='first')
print(len(dfb))

[1.0, 2.0, nan]
[2.0, 1.0, 3.0, nan]
   GDE_Class  GDE_Cert  Count_species  Count_plots
0        1.0       1.0            861        32391
1        1.0       2.0            601        21570
2        1.0       3.0            276         6237
3        2.0       1.0              3          323
4        2.0       2.0             53         1262
5        2.0       3.0             13          266
6        NaN       2.0              1          187
7        NaN       NaN          54240      1883128
56050
['Asia', 'Europe', 'Oceania', 'Africa', 'North America', 'South America']
               Species Continent  Count_plots  GDE_Class  GDE_Cert References  \
552  Breonia chinensis      Asia           11        1.0       2.0        NaN   
553  Breonia chinensis      Asia            2        1.0       1.0        NaN   

             ExpertName                ExpertEmail                  Notes  \
552  Kristina Toderich   ktoderich@tottori-u.ac.jp  Asia/mostly to China    
553  Kristina Toderich   k

In [61]:
# Read in plot data
df1 = pd.read_csv(plots_in, sep='\t')
df1

E:\Users\kklausmeyer\AppData\Local\Temp\36\ipykernel_14000\168297315.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(plots_in, sep='\t')


,PlotObservationID,GIVD_ID,Dataset,Continent,Country,Biome,Date_of_recording,Latitude,Longitude,Location_uncertainty,Releve_area,Plant_recorded,Elevation,Aspect,Slope,is_forest,ESY,Naturalness,Forest,Shrubland,Grassland,Wetland,Sparse_vegetation,Cover_total,Cover_tree_layer,Cover_shrub_layer,Cover_herb_layer,Cover_moss_layer,Cover_lichen_layer,Cover_algae_layer,Cover_litter_layer,Cover_bare_rocks,Cover_cryptogams,Cover_bare_soil,Height_trees_highest,Height_trees_lowest,Height_shrubs_highest,Height_shrubs_lowest,Height_herbs_average,Height_herbs_lowest,Height_herbs_highest,SoilClim_PC1,SoilClim_PC2,Resample_1,Resample_2,Resample_3,Resample_1_consensus
0,16,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,NaN,Not specified,1790.0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,30.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.660,0.546,True,False,False,True
1,17,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,NaN,Not specified,1750.0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.660,0.546,True,False,False,True
2,18,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,NaN,Not specified,1750.0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,5.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.660,0.546,False,False,True,False
3,20,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,NaN,Not specified,1520.0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,60.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.660,0.546,False,True,False,False
4,22,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,NaN,Not specified,1550.0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,65.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.660,0.546,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95099,1126749,EU-00-020,European wetlands,Europe,Latvia,Temperate midlatitudes,2010-08-19,56.287870,27.273862,10.0,4.0,Not specified,NaN,NaN,NaN,NaN,?,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,10.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.599,0.775,True,False,False,True
95100,1126761,EU-00-020,European wetlands,Europe,Latvia,Temperate midlatitudes,2009-06-19,56.939965,23.598014,10.0,4.0,Not specified,NaN,NaN,NaN,False,Y-Gra,Semi-natural,False,False,True,False,False,93.0,NaN,NaN,90.0,3.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.416,0.690,False,False,True,False
95101,1126769,EU-00-020,European wetlands,Europe,Latvia,Temperate midlatitudes,2009-06-19,56.939965,23.598014,10.0,4.0,Not specified,NaN,NaN,NaN,False,Y-Gra,Semi-natural,False,False,True,False,False,93.0,NaN,4.0,91.0,5.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.416,0.690,False,True,False,False
95102,1126774,EU-00-020,European wetlands,Europe,Latvia,Temperate midlatitudes,2003-09-06,57.248222,22.147667,10.0,4.0,Not specified,NaN,NaN,NaN,NaN,?,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,38.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.065,0.849,False,False,True,False


In [62]:
# Read in species data
df2 = pd.read_csv(veg_in, sep='\t')
df2

,PlotObservationID,Species,Original_species,Original_abundance,Abundance_scale,Relative_cover
0,16,Festuca brachyphylla,Festuca brachyphylla,10.0,CoverPerc,0.277778
1,16,Potentilla elegans,Potentilla elegans,25.0,CoverPerc,0.694444
2,16,Saxifraga serpyllifolia,Saxifraga serpyllifolia,1.0,CoverPerc,0.027778
3,17,Festuca brachyphylla,Festuca brachyphylla,1.0,CoverPerc,0.038462
4,17,Potentilla elegans,Potentilla elegans,25.0,CoverPerc,0.961538
...,...,...,...,...,...,...
1945379,1126790,Potentilla erecta,Potentilla erecta,0.3,CoverPerc,0.006276
1945380,1126790,Salix lapponum,Salix lapponum,1.0,CoverPerc,0.020921
1945381,1126790,Stellaria palustris,Stellaria palustris,0.5,CoverPerc,0.010460
1945382,1126790,Trichophorum alpinum,Scirpus hudsonianus,18.0,CoverPerc,0.376569


In [68]:
# Check species data from review table to ensure all have a match in the veg data
df3 = df2[['Species']].drop_duplicates()

# Reset the index of the new DataFrame
df3.reset_index(drop=True, inplace=True)
df3['sPlot'] = 1

df4 = pd.merge(dfb, df3, on='Species', how='left')
df5 = df4[df4['sPlot'].isna()]
df5

,Species,Continent,Count_plots,GDE_Class,GDE_Cert,References,ExpertName,ExpertEmail,Notes,Species_Kristina_values,Species_sPlot_values,Species_Kristina,Species_sPlot,sPlot
0,/new species,Asia,1,1.0,2.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,NaN,NaN,NaN,NaN,NaN,NaN
537,Bolboschoenus/salt marsh species,Asia,4,1.0,2.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
# Check the original data to see if those records exist
df3['Species'] = df3['Species'].fillna('')
df3[df3['Species'].str.startswith('Bolboschoenus')].sort_values(by='Species')

,Species,sPlot
28325,Bolboschoenus,1
3300,Bolboschoenus caldwellii,1
4965,Bolboschoenus fluviatilis,1
13875,Bolboschoenus glaucus,1
37379,Bolboschoenus laticarpus,1
18464,Bolboschoenus maritimus,1
13850,Bolboschoenus planiculmis,1
34905,Bolboschoenus robustus,1


In [55]:
# Optional:  Create a table to review the species names that don't match the values in the sPlot Open dataset

# Get the unique species names from df3
unique_species_df3 = df3['Species'].unique()

# Create a new DataFrame with the 'Species' column from df5
#df_closest_matches = df5[['Species']].head(5).copy()
df_closest_matches = df5[['Species']].copy()

# Rename the original 'Species' column to 'Species_df5'
df_closest_matches.rename(columns={'Species': 'Species_Kristina'}, inplace=True)

# Add a new empty 'Species_df3' column
df_closest_matches['Species_sPlot'] = ''

i=1
# Iterate through the rows of df_closest_matches and find the closest match
for index, row in df_closest_matches.iterrows():
    species_df5 = row['Species_Kristina']
    print("Finding match for record {} of {}, for species {}".format(i, len(df_closest_matches), species_df5))
    closest_match = process.extractOne(species_df5, unique_species_df3)[0]
    df_closest_matches.at[index, 'Species_sPlot'] = closest_match
    i+=1
# Display the DataFrame with the closest matches

df_closest_matches.to_csv(fuzzy_out, encoding='utf-8', index=False)

Finding match for record 1 of 138, for species Achillea asiatica Serg.
Finding match for record 2 of 138, for species Aegopodium podagraria/herbaceous long-rizomes perennial native to Eurasia.
Finding match for record 3 of 138, for species Agrimonia pilosa Ledeb,/perennial rizomatous herb
Finding match for record 4 of 138, for species Arisaema amurense/pereennial tuber plant
Finding match for record 5 of 138, for species Arachniodes aristata/perennial geophyte
Finding match for record 6 of 138, for species Angiopteris lygodiifolia/endangered species 
Finding match for record 7 of 138, for species Cenolophium denudatum/salt marshes
Finding match for record 8 of 138, for species Acantholimon subavenaceum  Lincz
Finding match for record 9 of 138, for species Acantholimon araxanum Bunge
Finding match for record 10 of 138, for species Alopecurus arundinaceus/rhizomatous perennial species 
Finding match for record 11 of 138, for species Acantholimon calvertii Boissier
Finding match for recor

Finding match for record 97 of 138, for species Bulbophyllum affine/grow in wet tropical forest
Finding match for record 98 of 138, for species Bursera fagaroides/dry deciduous desert shrub
Finding match for record 99 of 138, for species Cercocarpus macrophyllus/rizamatous perennial
Finding match for record 100 of 138, for species Chaerophyllum involucratum/perennial and usually densely tufted.

Finding match for record 101 of 138, for species Cyclocodon lancifolius/medicinal plant of high value 
Finding match for record 102 of 138, for species Cynanchum atratum/perennial herbaceous plant native to East Asia
Finding match for record 103 of 138, for species Acantholimon hissaricum Lincz
Finding match for record 104 of 138, for species Acantholimon zaprjagaevii Lincz
Finding match for record 105 of 138, for species Acinodendron pergamentaceum Kuntze.
Finding match for record 106 of 138, for species Aconitum lasiocarpum Rchb. 
Finding match for record 107 of 138, for species Adonis tiansc

PermissionError: [Errno 13] Permission denied: 'N:\\Projects\\Groundwater\\projects\\Global_GDE_Map\\tables\\sPlotOpen_fuzzy_match.csv'

In [79]:
# Join the plot data to the sPlot species table
df6 = pd.merge(df2, df1, on='PlotObservationID', how='left')
df6

,PlotObservationID,Species,Original_species,Original_abundance,Abundance_scale,Relative_cover,GIVD_ID,Dataset,Continent,Country,Biome,Date_of_recording,Latitude,Longitude,Location_uncertainty,Releve_area,Plant_recorded,Elevation,Aspect,Slope,is_forest,ESY,Naturalness,Forest,Shrubland,Grassland,Wetland,Sparse_vegetation,Cover_total,Cover_tree_layer,Cover_shrub_layer,Cover_herb_layer,Cover_moss_layer,Cover_lichen_layer,Cover_algae_layer,Cover_litter_layer,Cover_bare_rocks,Cover_cryptogams,Cover_bare_soil,Height_trees_highest,Height_trees_lowest,Height_shrubs_highest,Height_shrubs_lowest,Height_herbs_average,Height_herbs_lowest,Height_herbs_highest,SoilClim_PC1,SoilClim_PC2,Resample_1,Resample_2,Resample_3,Resample_1_consensus
0,16,Festuca brachyphylla,Festuca brachyphylla,10.0,CoverPerc,0.277778,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,NaN,Not specified,1790.0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,30.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.660,0.546,True,False,False,True
1,16,Potentilla elegans,Potentilla elegans,25.0,CoverPerc,0.694444,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,NaN,Not specified,1790.0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,30.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.660,0.546,True,False,False,True
2,16,Saxifraga serpyllifolia,Saxifraga serpyllifolia,1.0,CoverPerc,0.027778,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,NaN,Not specified,1790.0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,30.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.660,0.546,True,False,False,True
3,17,Festuca brachyphylla,Festuca brachyphylla,1.0,CoverPerc,0.038462,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,NaN,Not specified,1750.0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.660,0.546,True,False,False,True
4,17,Potentilla elegans,Potentilla elegans,25.0,CoverPerc,0.961538,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,NaN,Not specified,1750.0,NaN,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.660,0.546,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945379,1126790,Potentilla erecta,Potentilla erecta,0.3,CoverPerc,0.006276,EU-00-020,European wetlands,Europe,Latvia,Temperate midlatitudes,2003-07-19,57.563967,26.892074,10.0,4.0,Not specified,NaN,NaN,NaN,False,D32,Natural,False,True,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.738,1.164,False,False,True,False
1945380,1126790,Salix lapponum,Salix lapponum,1.0,CoverPerc,0.020921,EU-00-020,European wetlands,Europe,Latvia,Temperate midlatitudes,2003-07-19,57.563967,26.892074,10.0,4.0,Not specified,NaN,NaN,NaN,False,D32,Natural,False,True,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.738,1.164,False,False,True,False
1945381,1126790,Stellaria palustris,Stellaria palustris,0.5,CoverPerc,0.010460,EU-00-020,European wetlands,Europe,Latvia,Temperate midlatitudes,2003-07-19,57.563967,26.892074,10.0,4.0,Not specified,NaN,NaN,NaN,False,D32,Natural,False,True,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.738,1.164,False,False,True,False
1945382,1126790,Trichophorum alpinum,Scirpus hudsonianus,18.0,CoverPerc,0.376569,EU-00-020,European wetlands,Europe,Latvia,Temperate midlatitudes,2003-07-19,57.563967,26.892074,10.0,4.0,Not specified,NaN,NaN,NaN,False,D32,Natural,False,True,True,True,False,NaN,NaN,NaN,NaN

In [91]:
# Join the GDE review data to the sPlot data
df7 = pd.merge(dfb, df6, on=['Species', 'Continent'], how='inner')
df7 = df7.drop(['Species_sPlot_values', 'Species_Kristina', 'Species_sPlot'], axis=1)
df7 = df7.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'})

df7

,Species,Continent,Count_plots,GDE_Class,GDE_Cert,References,ExpertName,ExpertEmail,Notes,Species_Kristina_values,PlotObservationID,Original_species,Original_abundance,Abundance_scale,Relative_cover,GIVD_ID,Dataset,Country,Biome,Date_of_recording,latitude,longitude,Location_uncertainty,Releve_area,Plant_recorded,Elevation,Aspect,Slope,is_forest,ESY,Naturalness,Forest,Shrubland,Grassland,Wetland,Sparse_vegetation,Cover_total,Cover_tree_layer,Cover_shrub_layer,Cover_herb_layer,Cover_moss_layer,Cover_lichen_layer,Cover_algae_layer,Cover_litter_layer,Cover_bare_rocks,Cover_cryptogams,Cover_bare_soil,Height_trees_highest,Height_trees_lowest,Height_shrubs_highest,Height_shrubs_lowest,Height_herbs_average,Height_herbs_lowest,Height_herbs_highest,SoilClim_PC1,SoilClim_PC2,Resample_1,Resample_2,Resample_3,Resample_1_consensus
0,Abarema adenophora,Asia,2,1.0,1.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,"restricted to desert areas in central Asia, Af...",Abarema adenophora/rare species,56172,Abarema adenophora,13.0,x_SC,0.023091,00-00-003,Salvias,Indonesia,Tropics with year-round rain,NaN,-1.250000,110.170000,1000.0,1600.0,Woody plants >= 10 cm dbh,198.0,NaN,NaN,True,NaN,Natural,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.609,9.325,True,True,True,True
1,Abarema adenophora,Asia,2,1.0,1.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,"restricted to desert areas in central Asia, Af...",Abarema adenophora/rare species,58216,Abarema adenophora,100.0,x_SC,0.025641,00-00-003,Salvias,Malaysia,Tropics with year-round rain,NaN,3.956000,101.616000,100.0,100.0,Woody plants >= 2.5 cm dbh,890.0,NaN,NaN,True,NaN,Natural,True,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.642,8.230,True,True,True,True
2,Abelia corymbosa,Asia,2,1.0,1.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,Native to Central Asia,NaN,72629,Abelia corymbosa,2.0,CoverPerc,0.016949,AS-RU-004,Siberia_Kazachstan,Kazakhstan,Dry midlatitudes,2011-06-01,50.399496,70.079731,100.0,100.0,Not specified,330.0,360.0,0.0,False,NaN,NaN,False,False,True,False,False,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,20.0,-2.369,-1.091,False,True,False,False
3,Abelia corymbosa,Asia,2,1.0,1.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,Native to Central Asia,NaN,72638,Abelia corymbosa,2.0,CoverPerc,0.025641,AS-RU-004,Siberia_Kazachstan,Kazakhstan,Dry midlatitudes,2011-06-01,50.517016,70.153947,100.0,100.0,Not specified,359.0,360.0,0.0,False,NaN,NaN,False,False,True,False,False,NaN,NaN,NaN,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,10.0,-2.396,-1.118,True,True,True,True
4,Abelia mosanensis,Asia,1,1.0,2.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,evergreen shrubs are native to eastern Asia (J...,NaN,44006,Abelia mosanensis,2.0,CoverPerc,0.009662,AS-00-001,Korea,South Korea,Tropics with year-round rain,NaN,34.611111,127.433333,1000.0,225.0,Not specified,300.0,45.0,15.0,True,NaN,Semi-natural,True,False,False,False,False,NaN,85.0,25.0,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.637,3.516,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62004,Reaumuria soongarica,Asia,302,2.0,1.0,"Xu, H., Li, Y. Water-use strategy of three cen...",Kirk Klausmeyer,kklausmeyer@tnc.org,NaN,NaN,70962,Reaumuria songarica,5.0,CoverPerc,0.172414,AS-RU-002,Siberia,Russian Federation,Boreal zone,2007-06-21,50.026940,88.655800,10.0,100.0,Not specified,1807.0,360.0,0.0,NaN,NaN,Natural,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.267,-1.614,False,False,True,False
62005,Reaumuria soongarica,Asia,302,2.0,1.0,"Xu, H., Li, Y. Water-use strategy of three cen...",Kirk Klausmeyer,kklausmeyer@tnc.org,NaN,NaN,7

In [92]:
# Do another data check on the number of plots
df8 = (df7.groupby(['Species', 'Continent'], dropna=False)
           .agg({'PlotObservationID':'count'})
           .reset_index()
           .rename(columns={'PlotObservationID':'Count_plots_check'})
          )
df9 = pd.merge(df8, dfb, on=['Species', 'Continent'])
df9[df9['Count_plots_check'] != df9['Count_plots']]

,Species,Continent,Count_plots_check,Count_plots,GDE_Class,GDE_Cert,References,ExpertName,ExpertEmail,Notes,Species_Kristina_values,Species_sPlot_values,Species_Kristina,Species_sPlot
182,Aegilops neglecta,Asia,38,77,1.0,3.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,Europe,NaN,NaN,NaN,NaN
1792,Robinia pseudoacacia,Asia,17,4,1.0,1.0,NaN,Kristina Toderich,ktoderich@tottori-u.ac.jp,"Asia/Caucasus, Crimea and Central Asia",Acacia (Robinia pseudoacacia L.) white acacia,NaN,NaN,NaN


In [93]:
# Export table
df7.to_csv(tbl_out, encoding='utf-8', index=False)